# ChatbotService 코드 상세 분석

이 문서는 `chat/services.py` 파일의 구조와 로직을 상세하게 분석하고 설명합니다.

---

## 1. Import 및 경로 설정

Django 프로젝트 내에서 외부 모듈(`ai_module`)을 가져오기 위한 설정입니다.

In [ ]:
import sys
from pathlib import Path
from django.conf import settings

try:
    # ai_module이 Python 경로에 잡혀있다면 바로 import 시도
    from chat.ai_module.chatbot_graph_V8_FINAL import LegalRAGBuilder, Config
except ImportError:
    # 잡혀있지 않다면, 프로젝트 루트 경로를 sys.path에 추가하여 import 가능하게 함
    import sys
    sys.path.append(str(settings.BASE_DIR))
    from chat.ai_module.chatbot_graph_V8_FINAL import LegalRAGBuilder, Config

### 설명
- `try-except ImportError`: 개발 환경이나 배포 환경에 따라 `ai_module`의 경로 인식 여부가 다를 수 있어 예외 처리를 두었습니다.
- **`settings.BASE_DIR`**: Django 설정에 정의된 프로젝트 루트 디렉토리입니다. 이를 `sys.path`에 추가함으로써 파이썬이 해당 경로 아래의 모듈들을 찾을 수 있게 합니다.

## 2. ChatbotService 클래스 정의 (싱글톤 패턴)

`ChatbotService`는 챗봇 인스턴스를 하나만 생성하고 재사용하기 위해 **싱글톤 패턴(Singleton Pattern)**으로 구현되었습니다.

In [ ]:
class ChatbotService:
    _instance = None  # 싱글톤 인스턴스 저장
    _builder = None   # RAG 모델 빌더
    _graph = None     # 실행 가능한 LangGraph 객체
    
    @classmethod
    async def get_instance(cls):
        """비동기 방식으로 인스턴스를 가져오는 메서드"""
        if cls._instance is None:
            cls._instance = ChatbotService()
            await cls._instance._initialize()
        return cls._instance

### 핵심 포인트
- **`_instance`**: 이 변수에 생성된 유일한 객체를 저장합니다. 
- **`get_instance()`**: 
    - 인스턴스가 없으면 새로 만들고 초기화(`_initialize`)합니다.
    - 이미 있으면 저장된 인스턴스를 반환하여 중복 생성을 막습니다.
    - `async def`로 정의되어 비동기 환경(ASGI)에서 효율적으로 동작합니다.

## 3. 초기화 로직 (Sync vs Async)

초기화 메서드는 동기(`initialize`, `_sync_initialize`)와 비동기(`_initialize`) 두 가지 방식으로 구현되어 있습니다. 상황에 따라 적절한 것을 사용합니다.

In [ ]:
    @classmethod
    def initialize(cls):
        """서버 시작 시 호출되는 동기 초기화 메서드"""
        if cls._instance is None:
            cls._instance = ChatbotService()
            cls._instance._sync_initialize()
            
    def _sync_initialize(self):
        config = Config()  # 설정 로드
        self._builder = LegalRAGBuilder(config)  # 빌더 생성
        self._graph = self._builder.build()  # 그래프(챗봇 로직) 생성
        print("Chatbot Graph Initialized successfully (Sync).")


    async def _initialize(self):
        """비동기 초기화 메서드"""
        config = Config()
        self._builder = LegalRAGBuilder(config)
        self._graph = self._builder.build()
        print("Chatbot Graph Initialized successfully.")

### 상세 과정
1. **`Config()`**: 챗봇 설정(모델명, API 키 등)을 불러옵니다.
2. **`LegalRAGBuilder`**: RAG 시스템을 구축하는 빌더 객체입니다.
3. **`.build()`**: 실제 벡터 DB 연결, LLM 초기화, LangGraph 노드 연결 등을 수행하여 실행 가능한 그래프(`_graph`)를 만듭니다.
   - 이 과정이 무겁기 때문에 최초 1회만 실행하는 것이 성능상 중요합니다.

## 4. 답변 생성 메서드 (`get_response`)

사용자의 질문을 받아 LangGraph를 실행하고 결과를 반환하는 핵심 기능입니다.

In [ ]:
    async def get_response(self, user_message: str):
        # 혹시 그래프가 없으면 초기화 시도
        if not self._graph:
            await self._initialize()
            
        # LangGraph 입력값 구성
        inputs = {
            "user_query": user_message, 
            "messages": [("user", user_message)],
            "retry_count": 0
        }
        
        try:
            # ainvoke로 비동기 실행 (LangChain/LangGraph 메서드)
            result = await self._graph.ainvoke(inputs)
            
            # 결과에서 답변 추출
            return result.get("generated_answer", "죄송합니다. 답변을 생성하지 못했습니다.")
        except Exception as e:
            print(f"Error generation response: {e}")
            return f"오류가 발생했습니다: {str(e)}"

### 동작 흐름
1. **`inputs` 생성**: 사용자 질문(`user_query`)을 그래프 상태(State)에 맞는 형태로 포장합니다.
2. **`await self._graph.ainvoke(inputs)`**: 
   - 비동기로 그래프 로직(Search -> Analyze -> Generate 등)을 실행합니다.
   - `await`를 사용하여 실행되는 동안 서버가 멈추지 않고 다른 요청을 처리할 수 있게 합니다.
3. **결과 반환**: `generated_answer` 키에서 최종 답변을 꺼내 반환합니다. 예외 발생 시 에러 메시지를 반환합니다.